In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
BIT_CNT_EXT = 6

In [3]:
df = pd.read_csv("out.csv")
df.head()

,angle,sin,cos
0,0,0,3f
1,1,0,3f
2,2,1,3f
3,3,2,3f
4,4,3,3f


In [4]:
def ToDouble(x):
    result = 0.0
    x = bin(int(x, 16))[2:].zfill(BIT_CNT_EXT+2)
    temp = 1.0
    for bit in x[1:]:
        if bit == '1':
            result += temp
        temp = temp/2
    if x[0] == '1':
        result -= 2
    return result

for col in df:
    df["raw_" + str(col)] = df[col]
    df[col] = df[col].apply(ToDouble)
df.head()

,angle,sin,cos,raw_angle,raw_sin,raw_cos
0,0.000000,0.000000,0.984375,0,0,3f
1,0.015625,0.000000,0.984375,1,0,3f
2,0.031250,0.015625,0.984375,2,1,3f
3,0.046875,0.031250,0.984375,3,2,3f
4,0.062500,0.046875,0.984375,4,3,3f


In [5]:
def errF_ULP(angle, val, fun):
    ULP = 2**-BIT_CNT_EXT
    temp = fun(angle)
    err = np.abs(val - temp)
    ulpErr = err/ULP
    return ulpErr

def errF(angle, val, fun):
    temp = fun(angle)
    err = np.abs(val - temp)
    return err

df['ErrSin'] = df.apply(lambda x : errF(x.angle, x.sin, np.sin), axis=1)
df['ErrCos'] = df.apply(lambda x : errF(x.angle, x.cos, np.cos), axis=1)
df['ErrSin[ULP]'] = df.apply(lambda x : errF_ULP(x.angle, x.sin, np.sin), axis=1)
df['ErrCos[ULP]'] = df.apply(lambda x : errF_ULP(x.angle, x.cos, np.cos), axis=1)
df.head()

,angle,sin,cos,raw_angle,raw_sin,raw_cos,ErrSin,ErrCos,ErrSin[ULP],ErrCos[ULP]
0,0.000000,0.000000,0.984375,0,0,3f,0.000000,0.015625,0.000000,1.000000
1,0.015625,0.000000,0.984375,1,0,3f,0.015624,0.015503,0.999959,0.992188
2,0.031250,0.015625,0.984375,2,1,3f,0.015620,0.015137,0.999674,0.968753
3,0.046875,0.031250,0.984375,3,2,3f,0.015608,0.014527,0.998901,0.929700
4,0.062500,0.046875,0.984375,4,3,3f,0.015584,0.013673,0.997396,0.875041


In [7]:
df[['ErrSin[ULP]', 'ErrCos[ULP]', 'ErrSin', 'ErrCos']].describe(include='all')

,ErrSin[ULP],ErrCos[ULP],ErrSin,ErrCos
count,101.000000,101.000000,101.000000,101.000000
mean,0.603316,0.485556,0.009427,0.007587
std,0.307628,0.277389,0.004807,0.004334
min,0.000000,0.004376,0.000000,0.000068
25%,0.357473,0.251279,0.005586,0.003926
50%,0.630724,0.495775,0.009855,0.007746
75%,0.887896,0.689698,0.013873,0.010777
max,0.999959,1.000000,0.015624,0.015625
